## Extracción (Extract):
Lee el archivo JSON que contiene los datos crudos.

# **ETL raw-conversations**

In [3]:
import json

# Lee el archivo JSON
with open(r'C:\Users\wppaez\Desktop\hunty-etl\input\anonymized_raw_conversations_data.json', 'r') as file:
    data = json.load(file)

## Transformación (Transform):

Para crear una estructura tabular intuitiva y sencilla a partir de tus datos JSON, se considera crear las siguientes tablas relacionales:

### 1. Tabla de Conversaciones:

Esta tabla puede contener información sobre las conversaciones, como el ID de la conversación, el rol del participante (usuario, asistente, etc.) y el contenido del mensaje.

| ID Conversación | Rol          | Contenido                                                                                                                                                                                                                                                                                          |
|-----------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 673dc174bbdffa3d88a4c57a804781e98f86d00b730fa829d832b9753e7a3779 | user | Hola ! estoy interesado en la vacante [ID_VACANTE] !                                                                                                                                                                                                                       |
| 673dc174bbdffa3d88a4c57a804781e98f86d00b730fa829d832b9753e7a3779 | assistant | Hola, Te habla [NOMBRE_ASISTENTE] de recursos humanos de [NOMBRE_EMPRESA]. Para poder empezar esta conversación primero debes aceptar nuestros términos y condiciones de tratamiento de datos. [ENLACE_POLITICA_PRIVACIDAD]...|

### 2.Tabla de Usuarios

Esta tabla puede contener información sobre los usuarios participantes en las conversaciones, 

| ID Usuario | Nombre Completo | ID conversacion|
|------------|-----------------|----------------|
| [ID_USUARIO] | [NOMBRE_COMPLETO] | [APELLIDO] 


### 2.Tabla de Respuestas

Esta tabla contiene información de de las respuestas obtenidas de los usuarios a las preguntas claves del proceso contiene, el ID  de la conversadion, ID autoincremental del usuario, Tipo de pregunta y respuesta.
 
| ID Usuario | ID Coversacion       | Respuesta    | Tipo pregunta   |
|------------|----------------------|--------------|------------------
|[ID_USUARIO]| [ID CONVERSACION]    | [RESPUESTA]  | [TIPO]          |

## Ventajas de estructuracion de los datos

1.Organización clara de la información: Al dividir la información en tablas separadas, se logra una estructura organizada y fácil de entender, donde cada tabla tiene un propósito específico y contiene datos relacionados.

2.Reducción de la redundancia: La normalización de datos en varias tablas evita la repetición innecesaria de información, lo que optimiza el almacenamiento y reduce el riesgo de inconsistencias en los datos.

3.Integridad de los datos: Las tablas relacionadas permiten establecer relaciones y restricciones de integridad referencial, garantizando la coherencia y consistencia de los datos entre las diferentes entidades.

4.Eficiencia en consultas y análisis: La estructura de varias tablas facilita la realización de consultas y análisis más eficientes y precisos, ya que se pueden combinar datos de diferentes tablas mediante JOINs y otros operadores SQL.

5.Escalabilidad y flexibilidad: Esta estructura es escalable y flexible, lo que significa que se puede adaptar fácilmente a cambios y expansiones futuras en los datos y requisitos de análisis sin comprometer la integridad de la base de datos.

### Tabla Usuarios

In [87]:
import pandas as pd
import json

# Ruta del archivo JSON
file_path = 'C:/Users/wppaez/Desktop/hunty-etl/input/anonymized_raw_conversations_data.json'

# Cargar el archivo JSON
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Transformar los datos en una lista de diccionarios
conversaciones = []
for conversation_id, messages in data.items():
    if not messages:  # Verificar si la lista de mensajes está vacía
        conversacion = {
            'ID Conversación': conversation_id,
            'Rol': None,  # Colocar 'NULL' en lugar de None si se prefiere
            'Contenido': 'NULL'  # Colocar 'NULL' en lugar de None si se prefiere
        }
        conversaciones.append(conversacion)
    else:
        for message in messages:
            conversacion = {
                'ID Conversación': conversation_id,
                'Rol': message['role'].strip(),
                'Contenido': message['content'].strip()
            }
            conversaciones.append(conversacion)

# Crear un DataFrame de pandas con los datos de las conversaciones
df_conversaciones = pd.DataFrame(conversaciones)

# Guardar el DataFrame en un archivo Excel
excel_file_path = 'C:/Users/wppaez/Desktop/hunty-etl/output/conversaciones.xlsx'
df_conversaciones.to_excel(excel_file_path, index=False)

# Mostrar el DataFrame en consola
print(df_conversaciones)


                                       ID Conversación                Rol  \
0    673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...               user   
1    673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...          assistant   
2    673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...               user   
3    673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...          assistant   
4    673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...               user   
..                                                 ...                ...   
596  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...          assistant   
597  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...               user   
598  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...  generic_assistant   
599  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...               user   
600  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...  generic_assistant   

                                             Contenido  
0    Hola ! estoy 

In [7]:
from google.cloud import bigquery

# Especifica el ID de tu proyecto de Google Cloud Platform
project_id = 'tactical-gate-417815'

# Inicializa el cliente de BigQuery con el ID del proyecto
client = bigquery.Client(project=project_id)

# Nombre del dataset y tabla en BigQuery
dataset_id = 'hunty_001'
table_id = 'conversaciones'

# Crea el dataset si no existe
dataset = bigquery.Dataset(f"{client.project}.{dataset_id}")
try:
    client.create_dataset(dataset)
    print(f"Dataset {dataset_id} creado en BigQuery.")
except Exception as e:
    print(f"Error al crear el dataset: {e}")

# Configuración de carga de la tabla
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("ID_Conversacion", "STRING"),
        bigquery.SchemaField("Rol", "STRING"),
        bigquery.SchemaField("Contenido", "STRING"),
    ],
    # Modo de carga para reemplazar la tabla si existe
    write_disposition="WRITE_TRUNCATE",
)

table_ref = dataset.table(table_id)

# Carga el DataFrame en BigQuery
job = client.load_table_from_dataframe(
    df_conversaciones, table_ref, job_config=job_config
)
job.result()  # Espera a que se complete el job

print(f"DataFrame cargado en la tabla {dataset_id}.{table_id} en BigQuery.")


c:\Users\wppaez\Desktop\hunty-etl\venv\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Error al crear el dataset: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/tactical-gate-417815/datasets?prettyPrint=false: Already Exists: Dataset tactical-gate-417815:hunty_001
DataFrame cargado en la tabla hunty_001.conversaciones en BigQuery.


### Tabla de usuarios

In [89]:
import pandas as pd
import json

# Ruta del archivo JSON
file_path = 'C:/Users/wppaez/Desktop/hunty-etl/input/anonymized_raw_conversations_data.json'

# Cargar el archivo JSON
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Lista para almacenar las respuestas
usuarios = []

# Contador para el ID autoincremental
contador_id = 1

# Iterar sobre las conversaciones en el archivo JSON
for conversacion_id, mensajes in data.items():
    # Variables para almacenar temporalmente el nombre completo y el estado de búsqueda de la pregunta
    nombre_completo = ""
    buscando_nombre = False
    
    for mensaje in mensajes:
        if mensaje["role"].strip().lower() == "assistant" and ("cuál es tu nombre completo?" in mensaje["content"].strip().lower() or "indícame tu nombre completo" in mensaje["content"].strip().lower()):
            buscando_nombre = True
        elif buscando_nombre and mensaje["role"].strip().lower() == "user":
            nombre_completo = mensaje["content"].strip()
            buscando_nombre = False
    
    # Si se encontró un nombre completo, agregarlo a la lista de respuestas
    if nombre_completo:
        usuarios.append({
            "ID": contador_id,
            "Conversacion ID": conversacion_id,
            "Nombre Completo": nombre_completo
        })
        contador_id += 1
    else:
        usuarios.append({
            "ID": "NULL",
            "Conversacion ID": conversacion_id,
            "Nombre Completo": ""
        })

# Crear el DataFrame
df = pd.DataFrame(usuarios)

# Mostrar el DataFrame
print(df)

# Guardar el DataFrame en un archivo Excel
excel_file_path = 'C:/Users/wppaez/Desktop/hunty-etl/output/users.xlsx'
df.to_excel(excel_file_path, index=False)
print(f"Archivo Excel guardado en: {excel_file_path}")


      ID                                    Conversacion ID    Nombre Completo
0   NULL  673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...                   
1      1  5cd3713008635b158efa7406b25eabc6825e93a0625df6...  [NOMBRE_COMPLETO]
2      2  17fbd58ca771f94008ffb056f7226dde097a47039fb1ec...  [NOMBRE_COMPLETO]
3      3  cee4404e5f694ad6141cb1dda30f78df1a5ed3e0efa6de...  [NOMBRE_COMPLETO]
4   NULL  467b92deb304456a96d260db83d6fe421bcc30a5cf1113...                   
5      4  36f256f0c14392d372137b7874d33e347d0a9486830937...  [NOMBRE_COMPLETO]
6   NULL  cd00dc29b5c5e5fb660945d12255f9a110e4d643a73059...                   
7   NULL  60d1a00ead883aaca3c501c309e2cbf685f1cd61435241...                   
8   NULL  8e8aa543ee96e42cb85626d06d12995c5f93cf0b92057a...                   
9      5  417c5647e0cc51c44517b7bb1a44ba32a98a6bc78d17db...  [NOMBRE_COMPLETO]
10     6  2ef48296f91c4a0fa9e55128121dd1141c8ab9ce9d7b16...  [NOMBRE_COMPLETO]
11  NULL  b1eb364062dad98fbfa639a15f638f647b29fa286c

In [10]:
from google.cloud import bigquery

# Especifica el ID de tu proyecto de Google Cloud Platform
project_id = 'hunty-418216'

# Inicializa el cliente de BigQuery con el ID del proyecto
client = bigquery.Client(project=project_id)

# Nombre del dataset a crear
dataset_id = 'hunty_001'

# Crea el dataset si no existe
dataset = bigquery.Dataset(f"{client.project}.{dataset_id}")
try:
    client.create_dataset(dataset)
    print(f"Dataset {dataset_id} creado en BigQuery.")
except Exception as e:
    print(f"Error al crear el dataset: {e}")



Error al crear el dataset: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/hunty-418216/datasets?prettyPrint=false: Access Denied: Project hunty-418216: User does not have bigquery.datasets.create permission in project hunty-418216.


In [68]:
import pandas as pd
import json

# Ruta del archivo JSON
file_path = 'C:/Users/wppaez/Desktop/hunty-etl/input/anonymized_raw_conversations_data.json'

# Cargar el archivo JSON
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Lista para almacenar las respuestas
respuestas = []

# Contador para el ID autoincremental
contador_id = 1

# Iterar sobre las conversaciones en el archivo JSON
for conversacion_id, mensajes in data.items():
    # Variables para almacenar temporalmente las respuestas y el estado de búsqueda de las preguntas
    respuesta_ciudad = "NULL"
    respuesta_experiencia = "NULL"
    respuesta_procesos = "NULL"
    respuesta_educativo = "NULL"
    respuesta_motivacion = "NULL"
    buscando_respuesta_ciudad = False
    buscando_respuesta_experiencia = False
    buscando_respuesta_procesos = False
    buscando_respuesta_educativo = False
    buscando_respuesta_motivacion = False
    
    for mensaje in mensajes:
        if mensaje["role"].strip().lower() == "assistant" and "en qué ciudad vives" in mensaje["content"].strip().lower():
            buscando_respuesta_ciudad = True
        elif buscando_respuesta_ciudad and mensaje["role"].strip().lower() == "user":
            respuesta_ciudad = mensaje["content"].strip()
            buscando_respuesta_ciudad = False
        
        if mensaje["role"].strip().lower() == "assistant" and "si tienes experiencia en atención al público y manejo de documentos" in mensaje["content"].strip().lower() or "tienes experiencia en atención al público y manejo de documentos?" in mensaje["content"].strip().lower():
            buscando_respuesta_experiencia = True
        elif buscando_respuesta_experiencia and mensaje["role"].strip().lower() == "user":
            respuesta_experiencia = mensaje["content"].strip()
            buscando_respuesta_experiencia = False

        if mensaje["role"].strip().lower() == "assistant" and "acerca de tu experiencia en procesos de oficina, archivo y entrega de insumos y suministros" in mensaje["content"].strip().lower() or "más sobre tu experiencia en procesos de oficina, archivo y entrega de insumos y suministros" in mensaje["content"].strip().lower() or  "más detalles sobre tu experiencia en procesos de oficina, archivo y entrega de insumos y suministros" in mensaje["content"].strip().lower() :
            buscando_respuesta_procesos = True
        elif buscando_respuesta_procesos and mensaje["role"].strip().lower() == "user":
            respuesta_procesos = mensaje["content"].strip()
            buscando_respuesta_procesos = False

        if mensaje["role"].strip().lower() == "assistant" and "cuál es tu nivel educativo actual" in mensaje["content"].strip().lower():
            buscando_respuesta_educativo = True
        elif buscando_respuesta_educativo and mensaje["role"].strip().lower() == "user":
            respuesta_educativo = mensaje["content"].strip()
            buscando_respuesta_educativo = False

        if mensaje["role"].strip().lower() == "assistant" and "qué es lo que más te motiva de trabajar en" in mensaje["content"].strip().lower():
            buscando_respuesta_motivacion = True
        elif buscando_respuesta_motivacion and mensaje["role"].strip().lower() == "user":
            respuesta_motivacion = mensaje["content"].strip()
            buscando_respuesta_motivacion = False
    
    # Agregar las respuestas a la lista de respuestas con el tipo de pregunta
    respuestas.append({
        "ID": contador_id,
        "Conversación ID": conversacion_id,
        "ID Pregunta": 2,
        "Respuesta": respuesta_ciudad,
        "Tipo Pregunta": "excluyente"
    })
    contador_id += 1
    
    respuestas.append({
        "ID": contador_id,
        "Conversación ID": conversacion_id,
        "ID Pregunta": 3,
        "Respuesta": respuesta_experiencia,
        "Tipo Pregunta": "excluyente"
    })
    contador_id += 1
    
    respuestas.append({
        "ID": contador_id,
        "Conversación ID": conversacion_id,
        "ID Pregunta": 4,
        "Respuesta": respuesta_procesos,
        "Tipo Pregunta": "calificatoria"
    })
    contador_id += 1

    respuestas.append({
        "ID": contador_id,
        "Conversación ID": conversacion_id,
        "ID Pregunta": 5,
        "Respuesta": respuesta_educativo,
        "Tipo Pregunta": "calificatoria"
    })
    contador_id += 1

    respuestas.append({
        "ID": contador_id,
        "Conversación ID": conversacion_id,
        "ID Pregunta": 6,
        "Respuesta": respuesta_motivacion,
        "Tipo Pregunta": "calificatoria"
    })
    contador_id += 1

# Crear el DataFrame
df = pd.DataFrame(respuestas)

# Mostrar el DataFrame
print(df)

# Guardar el DataFrame en un archivo Excel
excel_file_path = 'C:/Users/wppaez/Desktop/hunty-etl/output/responses.xlsx'
df.to_excel(excel_file_path, index=False)
print(f"Archivo Excel guardado en: {excel_file_path}")


      ID                                    Conversación ID  ID Pregunta  \
0      1  673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...            2   
1      2  673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...            3   
2      3  673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...            4   
3      4  673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...            5   
4      5  673dc174bbdffa3d88a4c57a804781e98f86d00b730fa8...            6   
..   ...                                                ...          ...   
140  141  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...            2   
141  142  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...            3   
142  143  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...            4   
143  144  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...            5   
144  145  975cb0b63b0b7e0550268d4678c1abce25e3d689c612e3...            6   

                                             Respuesta  Tipo Pregunta  
0              